# Deep Q-Learning for Cart Pole (PyTorch)

In this notebook, we will use Deep Q-Learning to solve Cart Pole.


Environment: (https://gymnasium.farama.org/environments/classic_control/cart_pole/).

Paper: (https://arxiv.org/abs/1312.5602).

In [11]:
import gymnasium as gym
from aux_gym_functions import *
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm
# Virtual display
from pyvirtualdisplay import Display


## Observing the environment

## Setting hyperparameters and initializing an agent

In [12]:
import os
import pandas as pd
import json

results_PATH = "results"
PATH = os.path.join(results_PATH)
if not os.path.exists(PATH):
    os.makedirs(PATH)

## Playing and Training

In [13]:
new_line = {
    "name": None,
    "last_5%_train_reward": None,
    "last_5%_test_reward": None,
    "last_5%_length": None,
    "last_5%_training_error":None,
    "max_episodes": None,
    "start_epsilon": None,
    "final_epsilon": None,
    "learning_rate": None,
    "discount_factor": None,
    "double_q": None,
    "results_path": None,
}
df = pd.DataFrame(columns=list(new_line.keys()))

In [14]:
def simulate(params):
    test_seq = 0
    last_test_freq = 0 
    last_rewards = 0
    new_line["name"] = params[0]
    env = gym.make(new_line["name"], render_mode="rgb_array")
    new_line["learning_rate"] = params[2]
    new_line["max_episodes"] = params[3]
    new_line["start_epsilon"] = params[4]
    new_line["final_epsilon"] = params[5]
    new_line["discount_factor"] = params[6]
    new_line["double_q"] = params[1]

    epsilon_decay = new_line["start_epsilon"] / new_line["max_episodes"]
    batch_size = 64
    max_memory = 10000
    total_steps = 0
    test_freq = round(new_line["max_episodes"] * 0.025)
    tau = 0.01

    agent = DeepQLearningAgent(
        learning_rate=new_line["learning_rate"],
        initial_epsilon=new_line["start_epsilon"],
        epsilon_decay=epsilon_decay,
        final_epsilon=new_line["final_epsilon"],
        discount_factor=new_line["discount_factor"],
        batch_size=batch_size,
        max_memory=max_memory,
        observation_space=env.observation_space,
        action_space=env.action_space,
        is_double_network=new_line["double_q"],
        tau=tau,
    )

    folder_name = "-".join(
        [
            new_line["name"],
            str(new_line["max_episodes"]),
            str(round(new_line["start_epsilon"] * 100)),
            str(round(new_line["final_epsilon"] * 100)),
            str(round(new_line["learning_rate"] * 10**6)),
            str(round(new_line["discount_factor"] * 100)),
            str(new_line["double_q"]),
        ]
    )

    PATH = os.path.join(results_PATH, folder_name)
    if not os.path.exists(PATH):
        os.makedirs(PATH)

    # Create lists to contain total rewards, steps and accurracy per episode
    accuracies = []
    mean_rewards = []
    rewards = []
    steps = []

    steps_to_save_video = [0.1, 0.5, 1]
    episode_stop = [
        round(i * (new_line["max_episodes"] - 1)) for i in steps_to_save_video
    ]
    for episode_count in tqdm(range(new_line["max_episodes"])):
        state, info = env.reset()
        is_terminal = False
        total_reward = 0
        episode_step = 0

        # The Q-Network training
        while not is_terminal:
            episode_step += 1

            # Choose an action by greedily (with epsilon chance of random action) from the Q-Network
            action = agent.choose_action(state)

            # Play best or random move and get new state and reward from environment
            next_state, reward, terminated, truncated, info = env.step(action)
            is_terminal = terminated or truncated

            # Store experience
            agent.remember(state, action, reward, next_state, is_terminal)

            # Train neural network based on ReplayBuffer
            agent.update()

            # update the current observation
            state = next_state
            total_steps += 1
            total_reward += reward

        agent.decay_epsilon()

        mean_rewards.append(total_reward)
        if (episode_count + 1) % test_freq == 0:
            accur = test_accurracy(env, agent, env.spec.max_episode_steps, 25)
            if (
                (last_test_freq - np.mean(steps[:test_freq])) <= (np.mean(steps[:test_freq])*0.025)
                and len(accuracies) > 0
                and abs(accur - accuracies[-1]) <= (0.025 * accuracies[-1])
                and last_rewards - np.mean(rewards[-test_freq:]) <= (0.025 * last_rewards)
                and episode_count > new_line["max_episodes"]*0.5
            ):
                test_seq += 1
            else:
                test_seq = 0
            last_test_freq = np.mean(steps[:test_freq])
            last_rewards = np.mean(rewards[-test_freq:])
            accuracies.append(accur)

            print('step: %s, episode: %s, training reward mean: %s, test reward mean: %s, random move probability: %s' % (total_steps, episode_count+1, sum(mean_rewards)/test_freq, accur, agent.epsilon))
            mean_rewards.clear()

        rewards.append(total_reward)
        steps.append(episode_step)
        if episode_count in episode_stop:
            percentage = round((episode_count / new_line["max_episodes"]) * 100)
            video_name = os.path.join(
                results_PATH, folder_name, f"{percentage}.mp4"
            )
            record_trained_video(env, agent, video_name, env.spec.max_episode_steps)

        if test_seq == 10000000:
            break

    for file in os.listdir(os.path.join(results_PATH, folder_name)):
        if file.endswith(".json"):
            os.remove(os.path.join(os.path.join(results_PATH, folder_name), file))

    new_line["last_5%_train_reward"] = get_mean_last_5_percent(rewards)
    new_line["last_5%_test_reward"] = get_mean_last_5_percent(accuracies)
    new_line["last_5%_length"] = get_mean_last_5_percent(steps)
    new_line["last_5%_training_error"] = get_mean_last_5_percent(agent.training_error)
    new_line["results_path"] = os.path.join(results_PATH, folder_name)

    output = {'train_reward':rewards,
            'test_reward': accuracies,
            'episode_length':steps,
            'training_error':agent.training_error
    }
    #save output dict as pickle
    with open(os.path.join(results_PATH, folder_name, 'output.json'), 'w') as fp:
        json.dump(output, fp)
    df.loc[len(df)] = new_line
    df.to_csv(os.path.join(results_PATH, "results.csv"), index=False)
    env.close()
    return

In [15]:

params_list = [
    ["CartPole-v1", True, 1e-2, 10000, 1, .1, .99], 
    ["CartPole-v1", False, 1e-2, 10000, 1, .1, .99], 
    ["MountainCar-v0", True, 1e-2, 10000, 1, .1, .99], 
    ["MountainCar-v0", False, 1e-2, 10000, 1, .1, .99], 
    ["CartPole-v1", True, 1e-4, 10000, 1, .05, .99], 
    ["CartPole-v1", False, 1e-4, 10000, 1, .05, .99], 
    ["MountainCar-v0", True, 1e-4, 10000, 1, .05, .99], 
    ["MountainCar-v0", False, 1e-4, 10000, 1, .05, .99],
    ["CartPole-v1", True, 1e-6, 10000, 1, .01, .99], 
    ["CartPole-v1", False, 1e-6, 10000, 1, .01, .99], 
    ["MountainCar-v0", True, 1e-6, 10000, 1, .01, .99], 
    ["MountainCar-v0", False, 1e-6, 10000, 1, .01, .99]
] 
for params in params_list:
    simulate(params)

Double Deep Q-learning agent started with PyTorch


  3%|▎         | 255/10000 [00:10<09:26, 17.19it/s]

step: 5500, episode: 250, training reward mean: 22.0, test reward mean: 161.92, random move probability: 0.9750000000000028


  5%|▌         | 501/10000 [00:23<11:58, 13.21it/s]

step: 11336, episode: 500, training reward mean: 23.344, test reward mean: 121.88, random move probability: 0.9500000000000055


...

step: 1999984, episode: 10000, training reward mean: -200.0, test reward mean: -200.0, random move probability: 0.01


100%|█████████▉| 9999/10000 [41:07<00:00,  3.89it/s]

step 200 : 0 , [-0.70648825  0.00402056] , -1.0 , False , True , {}
Moviepy - Building video results/MountainCar-v0-10000-100-1-1-99-False/100.mp4.
Moviepy - Writing video results/MountainCar-v0-10000-100-1-1-99-False/100.mp4



100%|██████████| 10000/10000 [41:08<00:00,  4.05it/s]


Moviepy - Done !
Moviepy - video ready results/MountainCar-v0-10000-100-1-1-99-False/100.mp4
